# Chapter 15. Context Manager and else Blocks

Context managers may end up being almost as important as the subroutine itself. Basic has a `with` statement, there are `with` statements in lots of languages. But they don't do the same thing, they all do something very shallow, they save you from repeated dotted lookups, they don't do setup and tear down. The `with` satement is very big deal.

-- Raymod hettinger

The `with` statement sets up a tempory context and reliably tears it down, under the control of a context manager object. This prevent erros and reduce boilerplate code, making APIs at the same time safer and easier to use. Python programmers are finding lots of uses for with blocks beyond automatic file closing.

## Do This, Then That: else Blocks Beyond if

This is no secret, but it is an underappreciated language feature: the `else` clause can be used not only in `if` statement but also in `for`, `while`, and `try` statements.

The semantics of `for/else`, `while/else`, and `try/else` are closely related, but very different from `if/else`.

Here are the rules:

`for`
    
    The `else` block will run only if and when the for loop runs to completion(i.e., not if the for is aborted with a break)
    
`while`

    The else block will run only if and when the while loop exits because the condition become false.(i.e., not when the while is aborted with a break)
     
`try`

    The else block will only run if no exception is raised in the try block. The official docs state: "Excetpion in the else clause are not handled by the preceding except clause."

> I think `else` is very poor choice for the keyword in all cases except `if`. It implies an excluding alternative, like "Run this loop, otherwise do that", but the semantic for `else` in loops is the opposite: "Run this loop, then do that."

Using `else` with these statements often makes the code easier to read and saves the trouble of settings up control flags or adding extra if statements.

```python

for item in my_list:
    if item.flavor == 'banana':
        break
else:
    raise ValueError('No banana flavor found!')

```

In Python, `try/except` is commonly used for control flow, and not jsut for error handling. There's even an acronym/slogn for that docuemnt in the [official Python glossary](https://docs.python.org/3/glossary.html)

## Context Mangers and with Blocks

Context manager objects exits to control a `with` statement, just like iterators exits to control a `for` statement.

The `with` statement was designed to simplify the `try/finally` pattern, which guarantees that some operation is performed after a block of code, even if the block is aborted because of an exception, a return or `sys.exit()` call. The code in the finally clause usually releases a critical resource or restore some previous state that was tempoarily changed.

Th context manager protocol consists of the `__enter__` and `__exit__` methods. At the start of the with, `__enter__` is invoked on the context manager object. The role of the finnaly clause is played by a call ot `__exit__` on the context manager object at the end of the with block.

In [2]:
with open('mirror.py', 'w+') as fp:
    src = fp.read()

In [7]:
fp
# the fp variable is still available.

<_io.TextIOWrapper name='mirror.py' mode='w+' encoding='UTF-8'>

In [8]:
fp.closed, fp.encoding

# You can read the attributes of the fp object.

(True, 'UTF-8')

In [5]:
# fp.read(1)
# raise exception: I/O operation on closed file.

The open function returns an instance of `TextIOWrapper`, and its `__enter__` method returns self. But the `__enter__` method may also return some other object instance of the context manager.

When control flow exits the with block in any way, the `__exit__` method is invoked on the context manager object, **not on whatever is returnd by `__enter__`**

The `as` clause of the with statement is optional.

## Using @contextmanger

The `@contextmanager` decorator reduces the boilerplate of creating a context manager: instead of writing a whole class with `__enter__/__exit__` methods, you just implement a generator with a single yield that should produce whatever you want the `__enter__` method to return.

In a generator decorated with `@contextmanger`, `yield` is used to split the body of the function in two parts: everything before the yield will be executed at the beginning of the while blcok when the interpreter calls `__enter__`; the code after `yield` will run when `__exit__` is called at the end of the block.

In [10]:
import sys
import contextlib

In [11]:
@contextlib.contextmanager
def looking_glass():
    original_write = sys.stdout.write
    
    def reverse_write(text):
        original_write(text[::-1])
        
    sys.stdout.write = reverse_write
    
    yield "ABCDEDF"

    sys.stdout.write = original_write

In [13]:
with looking_glass() as s:
    print(s)

FDEDCBA


Essentially the `contextlib.contextmanager` decorator wraps the function in a class that implements the `__enter__` and `__exit__` methods.

The `__enter__` method of that class:

1. Invoking the generator and holds on to the generator object -- let's call it gen.
2. Calls next(gen) to make it run to the yield keyword.
3. Returns the value yielded by next(gen), so it can be bound to a target variable in the with/as form.

When the with block terminates, the `__exit__` method:

1. Checks an exception was passed as `exc_type`; if so, `gen.throw(exception)` is invoked, causing the exception to be raised in the yield line inside the generator function body.
2. Otherwise, next(gen) is called, resuming the execution of the generator function body after the yield.

If an exception is raised in the body of the `with` block, the Python interpreter will catch it and raise it again in the yield exception inside looking_glass.

## Chapter Summary

This chapter started easily enough with dicussion of else blocks in for, while, and try statement. Once you get used to the peculiar meaning of the else clause in these statements, I believe else can clarify your intentions.

We then covered context managers and the meaning of the with statement, quickly moving beyond its common use to automatically close opened files. We implemented a custom context manager: the LookingGlass class with the `__enter__/__exit__` methods, and saw how to handle exception in the `__exit__` method.

Finally, we reviewed functions in the contextlib standard library module. One of them, the `@contextmanager` decorator, makes it possible to implement a context manager using a single generator with one yield.